In [ ]:
from ipymlwidgets.widgets.image.image_grid import ImageGrid
from ipymlwidgets import Box
import numpy as np
import time
import ipywidgets as W

button = W.Button(description="Add")
button.on_click(lambda _: image.insert(0, np.random.randint(0,255,(10,10,3),dtype=np.uint8)))
display(button)

save_button = W.Button(description="Save")
save_button.on_click(lambda _: image.save_selected(f"./.temp/{time.time_ns()}"))
display(save_button)


info = W.Text()
display(info)

images = [np.ones((10, 10, 4),dtype=np.uint8) for _ in range(50)]
for i,image in enumerate(images):
    image[:,:,:-1] *= np.random.randint(0,255)
    image[:,:,3] *= 255


image = ImageGrid(images=images, cell_size=(8,8), rows=3)
image_grid = Box([image], layout={"display": "flex", "flex-direction": "column", "gap": "10px", "width": "50%"})

display(image_grid)

img = np.random.randint(0,255,(60,20,3),dtype=np.uint8)
image.append(img)  

def update_info(change):
    info.value = str(change['new'])
image.observe(update_info, names="selected")
